In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np
import skimage.io as io
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

In [2]:
import argparse
import glob
import json
import os
from pathlib import Path

import numpy as np
import torch
import yaml
from tqdm import tqdm

from utils.google_utils import attempt_load
from utils.datasets import create_dataloader
from utils.general import coco80_to_coco91_class, check_dataset, check_file, check_img_size, box_iou, \
    non_max_suppression, scale_coords, xyxy2xywh, xywh2xyxy, clip_coords, set_logging, increment_path
from utils.loss import compute_loss
from utils.metrics import ap_per_class
from utils.plots import plot_images, output_to_target
from utils.torch_utils import select_device, time_synchronized

from models.models import *

In [3]:
def load_classes(path):
    # Loads *.names file at 'path'
    with open(path, 'r') as f:
        names = f.read().split('\n')
    return list(filter(None, names))  # filter removes empty strings (such as last line)

In [14]:
parser = argparse.ArgumentParser(prog='test.py --conf-thres 0.5 --img 416 --batch 32 --device cpu --data zero-waste-1/data.yaml --cfg yolor/cfg/yolor_p6.cfg --weights "runs/train/yolor_p6_2022_04_05-19_58_08/weights/best_overall.pt" --task test --names yolor/data/zerowaste.names --verbose --save-json --save-conf')
parser.add_argument('--weights', nargs='+', type=str, default="../runs/train/yolor_p6_2022_04_05-19_58_08/weights/best_overall.pt", help='model.pt path(s)')
parser.add_argument('--data', type=str, default="../zero-waste-1/data.yaml", help='*.data path')
parser.add_argument('--batch-size', type=int, default=32, help='size of each image batch')
parser.add_argument('--img-size', type=int, default=448, help='inference size (pixels)')
parser.add_argument('--conf-thres', type=float, default=0.5, help='object confidence threshold')
parser.add_argument('--iou-thres', type=float, default=0.65, help='IOU threshold for NMS')
parser.add_argument('--task', default='test', help="'val', 'test', 'study'")
parser.add_argument('--device', default='cpu', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
parser.add_argument('--single-cls', action='store_true', help='treat as single-class dataset')
parser.add_argument('--augment', action='store_true', help='augmented inference')
parser.add_argument('--verbose', default='True', action='store_true', help='report mAP by class')
parser.add_argument('--save-txt',  action='store_true', help='save results to *.txt')
parser.add_argument('--save-conf', default='True', action='store_true', help='save confidences in --save-txt labels')
parser.add_argument('--save-json', default='True',action='store_true', help='save a cocoapi-compatible JSON results file')
parser.add_argument('--project', default='runs/test', help='save to project/name')
parser.add_argument('--name', default='exp', help='save to project/name')
parser.add_argument('--exist-ok', action='store_true', help='existing project/name ok, do not increment')
parser.add_argument('--cfg', type=str, default='cfg/yolor_p6.cfg', help='*.cfg path')
parser.add_argument('--names', type=str, default='data/zerowaste.names', help='*.cfg path')
opt = parser.parse_args()
opt.save_json |= opt.data.endswith('coco.yaml')
opt.data = check_file(opt.data)  # check file
print(opt)

usage: test.py --conf-thres 0.5 --img 416 --batch 32 --device cpu --data zero-waste-1/data.yaml --cfg yolor/cfg/yolor_p6.cfg --weights "runs/train/yolor_p6_2022_04_05-19_58_08/weights/best_overall.pt" --task test --names yolor/data/zerowaste.names --verbose --save-json --save-conf
       [-h] [--weights WEIGHTS [WEIGHTS ...]] [--data DATA]
       [--batch-size BATCH_SIZE] [--img-size IMG_SIZE]
       [--conf-thres CONF_THRES] [--iou-thres IOU_THRES] [--task TASK]
       [--device DEVICE] [--single-cls] [--augment] [--verbose] [--save-txt]
       [--save-conf] [--save-json] [--project PROJECT] [--name NAME]
       [--exist-ok] [--cfg CFG] [--names NAMES]
test.py --conf-thres 0.5 --img 416 --batch 32 --device cpu --data zero-waste-1/data.yaml --cfg yolor/cfg/yolor_p6.cfg --weights "runs/train/yolor_p6_2022_04_05-19_58_08/weights/best_overall.pt" --task test --names yolor/data/zerowaste.names --verbose --save-json --save-conf: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --c

SystemExit: 2

In [11]:
data = "../zero-waste-1/data.yaml"
weights="../runs/train/yolor_p6_2022_04_05-19_58_08/weights/best_overall.pt",
batch_size=16,
imgsz=640,
conf_thres=0.5,
iou_thres=0.6,  # for NMS
save_json=True,
single_cls=False,
augment=False,
verbose=True,
model=None,
dataloader=None,
save_dir=Path(''),  # for saving images
save_txt=False,  # for auto-labelling
save_conf=False,
plots=True,
log_imgs=0  # number of logged images
task = "test"

In [4]:
from argparse import Namespace
opt = Namespace(augment=False, batch_size=32, cfg='./cfg/yolor_p6.cfg', conf_thres=0.5, data='../zero-waste-1/data.yaml', device='cpu', exist_ok=False, img_size=416, iou_thres=0.65, name='exp', names='./data/zerowaste.names', project='../runs/test', save_conf=True, save_json=True, save_txt=False, single_cls=False, task='test', verbose=True, weights=['../runs/train/yolor_p6_2022_04_05-19_58_08/weights/best_overall.pt'])

In [49]:
augment=False
batch_size=32
cfg='cfg/yolor_p6.cfg'
conf_thres=0.5
data='../zero-waste-1/data.yaml'
device='cpu'
exist_ok=False
img_size=416
iou_thres=0.65
name='exp'
names='data/zerowaste.names'
project='../runs/test'
save_conf=True
save_json=True
save_txt=False
single_cls=False
task='test'
verbose=True
weights=['../runs/train/yolor_p6_2022_03_26-10_44_07/weights/best_overall.pt']
imgsz=448

In [43]:
set_logging()
device = select_device(opt.device, batch_size=batch_size)
save_txt = opt.save_txt  # save *.txt labels

# Directories: where all the logging file will be saved for this experiment
save_dir = Path(increment_path(Path(opt.project) / opt.name, exist_ok=opt.exist_ok))  # increment run
(save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

# Load model
model = Darknet(opt.cfg).to(device)
try:
    ckpt = torch.load(weights[0], map_location=device)  # load checkpoint
    ckpt['model'] = {k: v for k, v in ckpt['model'].items() if model.state_dict()[k].numel() == v.numel()}
    model.load_state_dict(ckpt['model'], strict=False)
except:
    load_darknet_weights(model, weights[0])
imgsz = check_img_size(imgsz, s=64)  # check img_size

# Half: half precision if we are running on GPU
half = device.type != 'cpu'
if half:
    model.half()



Using torch 1.9.0+cu111 CPU

Model Summary: 665 layers, 36854616 parameters, 36854616 gradients, 80.410315000 GFLOPS


In [53]:
!pwd

/projectnb2/dl523/students/dong760/roboflow-ai/yolor


In [58]:
# Configure
yaml_dir='/projectnb2/dl523/students/dong760/roboflow-ai/zero-waste-1/data.yaml'
model.eval()
is_coco = yaml_dir.endswith('coco.yaml')  # is COCO dataset
with open(yaml_dir) as f:
    data = yaml.load(f, Loader=yaml.FullLoader)  # model dict


In [59]:
data

{'names': ['cardboard', 'metal', 'rigid_plastic', 'soft_plastic'],
 'nc': 4,
 'train': '/projectnb2/dl523/students/dong760/roboflow-ai/zero-waste-1/train/images',
 'val': '/projectnb2/dl523/students/dong760/roboflow-ai/zero-waste-1/valid/images',
 'test': '/projectnb2/dl523/students/dong760/roboflow-ai/zero-waste-1/test/images'}

In [60]:
check_dataset(data)  # check
nc = 1 if single_cls else int(data['nc'])  # number of classes
iouv = torch.linspace(0.5, 0.95, 10).to(device)  # iou vector for mAP@0.5:0.95
niou = iouv.numel()

In [61]:
# Logging
log_imgs, wandb = min(log_imgs, 100), None  # ceil
try:
    import wandb  # Weights & Biases
except ImportError:
    log_imgs = 0

# Dataloader
img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
_ = model(img.half() if half else img) if device.type != 'cpu' else None  # run once
path = data['test'] if task == 'test' else data['val']  # path to val/test images
dataloader = create_dataloader(path, imgsz, batch_size, 64, opt, pad=0.5, rect=True)[0]

NameError: name 'log_imgs' is not defined

In [9]:
anno_json = glob.glob("/projectnb/dl523/students/dong760/roboflow-ai/test/_annotations.coco.json")[0]
# importing the module
pred_json = "../runs/test/exp43/best_overall_predictions.json"
import json
 
# Opening JSON file
f1 = open(anno_json)
f2 = open(pred_json)
# returns JSON object as
# a dictionary
gT_dict = json.load(f1)
jdict = json.load(f2)




In [ ]:
converer_dict = gT_dict["images"]
suffix = converer_dict[0]["file_name"].split('.')[-1]
for i in range(len(jdict)):
    image_id = jdict[i]['image_id'] + "." + suffix
    if not image_id.isnumeric():
        flag = True # Mark as incorrect image_id
    for item in converer_dict:
        # If we found a match, and the current image_id is marked as incorrect format ==> We will correct it
        if flag and image_id == item["file_name"]:
            print(f"jdict[i]['image_id']: {jdict[i]['image_id']}, item['id']: {item['id']}")
            jdict[i]['image_id'] = item['id']

In [24]:
jdict[1]["image_id"].isnumeric()

'08_frame_011500_PNG.rf.fa2aa81dc2839b709ca7ce5e9feeed9b'

In [26]:

converer_dict[428]["file_name"].split('.')[-1]
# for item in converer_dict:
#     # If we found a match, and the current image_id is marked as incorrect format ==> We will correct it
#     if flag and image_id == item["file_name"]:
#         jdict[i]["image_id"] = item["id"]

'jpg'

In [31]:
pred_json = "../runs/test/exp51/best_overall_predictions.json"
# anno_json = glob.glob("coco/annotations/instances_val*.json")[0]  
anno_json = glob.glob("/projectnb2/dl523/students/dong760/zerowaste_dataset/zerowaste-f-final/test/labels.json")[0]  # finding the annotations json 
# anno_json = glob.glob("/projectnb/dl523/students/dong760/roboflow-ai/test/_annotations.coco.json")[0]  # finding the annotations json 
is_coco = False
try:  # https://github.com/cocodataset/cocoapi/blob/master/PythonAPIpycocoEvalDemo.ipynb
    from pycocotools.coco import COCO
    from pycocotools.cocoeval import COCOeval
    anno = COCO(anno_json)  # init annotations api
    pred = anno.loadRes(pred_json)  # init predictions api
    # imgIds=sorted(anno.getImgIds())
    # imgIds=imgIds[0:100]
    # imgId = imgIds[np.random.randint(100)]
    eval = COCOeval(anno, pred, 'bbox')
    if is_coco:
        eval.params.imgIds = [int(Path(x).stem) for x in dataloader.dataset.img_files]  # image IDs to evaluate
    eval.evaluate()
    eval.accumulate()
    eval.summarize()
    map, map50 = eval.stats[:2]  # update results (mAP@0.5:0.95, mAP@0.5)
    print("================> Start Printing evaluated result:")
    print(eval.stats)
    print("================> End of Printing!")
except Exception as e:
    print('ERROR: pycocotools unable to run: %s' % e)

loading annotations into memory...
Done (t=0.22s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.45s).
Accumulating evaluation results...
DONE (t=0.11s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

In [79]:
anno.getCatIds()

[1, 2, 3, 4]

In [3]:
#initialize COCO ground truth api
dataDir='../'
dataType='val2014'
annFile = '%s/annotations/%s_%s.json'%(dataDir,prefix,dataType)
cocoGt=COCO(annFile)

loading annotations into memory...
Done (t=8.01s)
creating index...
index created!


In [4]:
#initialize COCO detections api
resFile='%s/results/%s_%s_fake%s100_results.json'
resFile = resFile%(dataDir, prefix, dataType, annType)
cocoDt=cocoGt.loadRes(resFile)

Loading and preparing results...     
DONE (t=0.05s)
creating index...
index created!


In [5]:
imgIds=sorted(cocoGt.getImgIds())
imgIds=imgIds[0:100]
imgId = imgIds[np.random.randint(100)]

In [6]:
# running evaluation
cocoEval = COCOeval(cocoGt,cocoDt,annType)
cocoEval.params.imgIds  = imgIds
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

Running per image evaluation...      
DONE (t=0.46s).
Accumulating evaluation results...   
DONE (t=0.38s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.505
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.697
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.573
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.586
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.519
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.501
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.387
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.594
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.595
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.640
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.566
 Average Rec